<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/fr_whisper_medium_test_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8slf6rab
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-8slf6rab
  Resolved https://github.com/huggingface/transformers to commit df5c5c62ae253055336f5bb0828ca8e3e15ab6bd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8195433 sha256=3d32b831d53e498458e8e6a42851e32fe0eeb3a9cbf5dd794d93b4ad3851c943
  Stored in directory: /tmp/pip-ephem-wheel-cache-4il2r9r7/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "fr", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1742
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=59620bcf898f6885355f595f025f6c782e25000764520dd2fd2daf87718fb042
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [31]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:26<00:00, 58.4MiB/s]


In [32]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [33]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [34]:
text

,0
0,je souhaite juste rappeler que ces droits de p...
1,les parlements nationaux ont accepté que les c...
2,c'est ce que je demande à chacun d'entre vous ...
3,monsieur le président mesdames et messieurs le...
4,autrement dit il faut garantir à tous l'access...
...,...
1737,l'adoption de rapports par le parlement europé...
1738,monsieur le président le très bon rapport de m.
1739,mettons le en œuvre où il s'avère nécessaire.
1740,c'est du moins ce que je souhaite que nous con...


In [35]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [36]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,je souhaite juste rappeler que ces droits de p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0184326171875, -0.005706787109375, 0.01486...",16000
1,female,les parlements nationaux ont accepté que les c...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00091552734375, 0.0015869140625, -0.0005798...",16000
2,female,c'est ce que je demande à chacun d'entre vous ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000701904296875, -0.000732421875, -0.001312...",16000
3,male,monsieur le président mesdames et messieurs le...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00054931640625, 0.0009765625, -0.000427246...",16000
4,female,autrement dit il faut garantir à tous l'access...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005523681640625, 0.006256103515625, 0.00128...",16000
...,...,...,...,...,...
1737,female,l'adoption de rapports par le parlement europé...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0137939453125, 0.003936767578125, -0.002288...",16000
1738,male,monsieur le président le très bon rapport de m.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, 0.00115966796875, -0.00094...",16000
1739,male,mettons le en œuvre où il s'avère nécessaire.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00140380859375, 0.000274658203125, 0.00100...",16000
1740,female,c'est du moins ce que je souhaite que nous con...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.070068359375, 0.05706787109375, 0.054595947...",16000


In [37]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,je souhaite juste rappeler que ces droits de p...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0184326171875, -0.005706787109375, 0.01486...",16000
3,male,monsieur le président mesdames et messieurs le...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00054931640625, 0.0009765625, -0.000427246...",16000
5,male,l'efficacité passe aussi par un système de méc...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004974365234375, 0.00189208984375, 0.000457...",16000
7,male,je remercie encore le parlement européen et to...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000213623046875, -0.00042724609375, 0.0001...",16000
8,male,nous avons rappelé l'importance d'un contrôle ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000579833984375, -0.00054931640625, 0.00119...",16000
...,...,...,...,...,...
1734,male,monsieur le commissaire monsieur le président ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003173828125, 9.1552734375e-05, 0.001556396...",16000
1735,male,bien sûr j'en conviens coordonner les politiqu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0003662109375, 6.103515625e-05, 0.000122070...",16000
1738,male,monsieur le président le très bon rapport de m.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, 0.00115966796875, -0.00094...",16000
1739,male,mettons le en œuvre où il s'avère nécessaire.,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00140380859375, 0.000274658203125, 0.00100...",16000


In [38]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,les parlements nationaux ont accepté que les c...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00091552734375, 0.0015869140625, -0.0005798...",16000
2,female,c'est ce que je demande à chacun d'entre vous ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000701904296875, -0.000732421875, -0.001312...",16000
4,female,autrement dit il faut garantir à tous l'access...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005523681640625, 0.006256103515625, 0.00128...",16000
6,female,monsieur le président à mon tour de m'inquiéte...,/root/.cache/huggingface/datasets/downloads/ex...,"[-9.1552734375e-05, 3.0517578125e-05, -0.00024...",16000
15,female,pour bien peser il nous faut une stratégie vis...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00042724609375, 0.00018310546875, 0.0001831...",16000
...,...,...,...,...,...
1728,female,cet accord était l'occasion de gommer l'impact...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000701904296875, -0.000518798828125, -0.00...",16000
1729,female,mais avant cela il faut rendre les programmes ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00177001953125, -0.00244140625, -0.00683593...",16000
1736,female,nous avons déjà l'année dernière adopté une ré...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000518798828125, 0.000946044921875, -0.0009...",16000
1737,female,l'adoption de rapports par le parlement europé...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0137939453125, 0.003936767578125, -0.002288...",16000


In [39]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="fr")
    return result["text"]

In [40]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [41]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.2909090909090909
Processed 2 files, WER: 0.7692307692307693
Processed 3 files, WER: 0.35714285714285715
Processed 4 files, WER: 0.23076923076923078
Processed 5 files, WER: 0.0
Processed 6 files, WER: 0.21739130434782608
Processed 7 files, WER: 0.23076923076923078
Processed 8 files, WER: 0.15789473684210525
Processed 9 files, WER: 0.27586206896551724
Processed 10 files, WER: 0.2727272727272727
Processed 11 files, WER: 0.22
Processed 12 files, WER: 0.0
Processed 13 files, WER: 0.11764705882352941
Processed 14 files, WER: 0.24
Processed 15 files, WER: 0.5333333333333333
Processed 16 files, WER: 0.0
Processed 17 files, WER: 0.045454545454545456
Processed 18 files, WER: 0.07142857142857142
Processed 19 files, WER: 0.07692307692307693
Processed 20 files, WER: 0.30434782608695654
Processed 21 files, WER: 0.2413793103448276
Processed 22 files, WER: 0.22727272727272727
Processed 23 files, WER: 0.06666666666666667
Processed 24 files, WER: 0.19047619047619047
Processed 2

In [42]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.0
Processed 2 files, WER: 0.2
Processed 3 files, WER: 0.5121951219512195
Processed 4 files, WER: 0.36585365853658536
Processed 5 files, WER: 0.45454545454545453
Processed 6 files, WER: 0.27586206896551724
Processed 7 files, WER: 0.2777777777777778
Processed 8 files, WER: 0.0
Processed 9 files, WER: 0.2571428571428571
Processed 10 files, WER: 0.38095238095238093
Processed 11 files, WER: 0.625
Processed 12 files, WER: 0.12280701754385964
Processed 13 files, WER: 0.14285714285714285
Processed 14 files, WER: 0.3333333333333333
Processed 15 files, WER: 0.4
Processed 16 files, WER: 0.22033898305084745
Processed 17 files, WER: 0.2
Processed 18 files, WER: 0.29411764705882354
Processed 19 files, WER: 0.2571428571428571
Processed 20 files, WER: 0.13043478260869565
Processed 21 files, WER: 0.3333333333333333
Processed 22 files, WER: 0.09090909090909091
Processed 23 files, WER: 0.16216216216216217
Processed 24 files, WER: 0.35294117647058826
Processed 25 files, WER: 0.25